## Лаба 4. Прогнозирование пола и возрастной категории — Spark Streaming

#### Дедлайн

Четверг, 3 ноября 2022 до 12:00 МСК

#### Задача

У вас имеются данные: логи посещения пользователей по разным сайтам рунета. По этим пользователям вам также известны: пол и возрастная категория. Вам нужно будет в real-time спрогнозировать эти характеристики по пользователям, о которых у вас нет этой информации, но будут все те же самые логи посещения.

##### При решении задачи запрещено использовать библиотеки pandas, sklearn (кроме sklearn.metrics), xgboost и прочие.

#### Описание данных

Обучающая выборка, с которой вы будете работать, выглядит следующим образом:

```json
gender	age	uid	user_json
F	18-24	d50192e5-c44e-4ae8-ae7a-7cfe67c8b777	{"visits": [{"url": "http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun", "timestamp": 1419688144068}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426666298001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426666298000}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426661722001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426661722000}]}
```

Она расположена на hdfs: `/labs/slaba04/`.

Поле `gender` принимает значения `F` (женщина) и `M` (мужчина).

Поле `age` принимает значения диапазона возраста: `18-24`, `25-34`, `35-44`, `45-54`, `>=55`

Поле `uid` принимает значения уникального ID пользователя (cookies): `d50192e5-c44e-4ae8-ae7a-7cfe67c8b777`.

Поле `user_json` имеет внутри json со следующей схемой данных: `{"visits": [{"url": "url1", "timestamp": "timestamp1"}, {"url": "url2", "timestamp": "timestamp2"}]}`. В нем содержатся непосредственно логи посещения пользователем страниц вместе с временной меткой посещения.

Данные, по которым вам надо будет построить прогноз (тестовая выборка), хранятся в Kafka в таком виде:

```json
{"uid": "bdd48781-8243-493d-8ae6-794050a4417f",
 "visits": "[{"url": "http://vk.com/feed", "timestamp": 1425077901001}, {"url": "http://big-cards.a5ltd.com/app/html/vk.html?api_url=http://api.vk.com/api.php&api_id=1804162&api_settings=663823&viewer_id=40849488&viewer_type=2&sid=1b77e9ca975573fed6e31746ca58ea2509f3588d918e7dab2a291f7f7579ae7dab6c21058c837b74b1fbb&secret=37339dfcbf&access_token=06c53bd8f9f9a0afad23b4e166e448495396259fc5a1c5e5243474df82591bd5d0afe877d993ea4a82a81&user_id=40849488&group_id=0&is_app_user=1&auth_key=d7c91ad4bb27e20cb3ed81810f5649db&language=0&parent_language=0&ad_info=elsdcqvcsvftaqxtawjsxht b0q8htjxuvbbjrvbnwojfji2ha8h&is_secure=0&ads_app_id=1804162_903aa103fa48b1e378&referrer=menu&lc_name=2c073eed&hash=", "timestamp": 1425077901000}]"}
```

То есть все то же самое, только без поля `gender` и `age` и немного с другой схемой данных. Название топика, в который мы будем присылать данные: `input_ivan.ivanov`, где вместо `ivan.ivanov` ваш логин от личного кабинета. Данные вам будут поступать по нажатию кнопки "Проверить" в личном кабинете. Вам топик создавать не нужно. Он будет автоматом создаваться при записи. Чистить его тоже не надо, потому что Spark Streaming хранит оффсеты, то есть знает, что он уже считывал, что еще не считывал.

Порт брокера Kafka: `6667`. Hostname: `spark-master-1.newprolab.com` или `spark-node-1.newprolab.com`.

#### Результат

Вам в свою очередь нужно будет, считывая данные из Kafka, делать прогноз по ним в Spark Streaming и отправлять в real-time эти прогнозные значения в свой другой топик в Kafka в формате:

```json
{"uid": "fe1dba8f-3131-439f-9031-851c0da0f126", "gender": "M", "age": "25-34"}
{"uid": "d50192e5-c44e-4ae8-ae7a-7cfe67c8b777", "gender": "F", "age": "18-24"}
```

Название топика — ваш логин в личном кабинете: `ivan.ivanov`. После отправки данных чекер будет ждать 90 секунд, чтобы вы обработали входные данные из топика `input_ivan.ivanov` и записали результаты прогноза уже в этот топик.

#### Подсказки

Чтение из Kafka:

```python
read_kafka_params = {
    "kafka.bootstrap.servers": 'spark-master-1.newprolab.com:6667',
    "subscribe": "input_ivan.ivanov",
    "startingOffsets": "latest"
}
kafka_sdf = spark.readStream.format("kafka").options(**read_kafka_params).load()
```

Запись в Kafka:

```python
write_kafka_params = {
   "kafka.bootstrap.servers": 'spark-master-1.newprolab.com:6667',
   "topic": "ivan.ivanov"
}
batch_df.writeStream.format("kafka").options(**write_kafka_params)\
    .option("checkpointLocation", "streaming/chk/chk_kafka")\
    .outputMode("append").start()
```

Бывают проблемы при чтении старых данных, которых уже нет в кафке:

```
.option("failOnDataLoss", 'False')
```

#### Проверка
Проверка лабы устроена следующим образом:
1. Вы готовите свою модель
2. Настраиваете чтение данных из одного топика и запись в другой топик.
4. Запускаете свой скрипт в стриминговым режиме
3. Идет подключение к топику на чтение данных.
4. На странице лабы нажимаете кнопку "Проверить".
5. Вам в топик текут данные.
6. Модель обрабатывает входные данные.
7. Ваш скрипт сохраняет данные в другой топик.
8. Чекер на странице лабы выдает результат проверки.


Проверка осуществляется автоматическим скриптом на странице лабы в личном кабинете. Наш чекер в конечном итоге подключится к вашему финальному топику в Kafka и сравнит ваш прогноз с эталонным ответом. На выходе вы получите свой score. В качестве него метрика `accuracy`: доля пользователей от общего числа, по которым вы верно угадали и пол, и возрастную категорию. Если `accuracy` будет больше **0.25**, то лаба будет засчитана.

##### Особенности проверки
1. Проверка (генерация событий, обработка вашим скриптом, чтение и проверка результатов) занимают до 90 секунд, просьба ожидать (можно нажимать кнопку "обновить")
2. За один раз (одно нажание "Проверить") чекер присылает батч в 5000 событий. Если для отладки / повторной проверки требуется еще, надо нажимать снова

####  Рекомендации по прохождению
1. Разработанную модель (и вообще весь пайплайн) можно сохранить в HDFS и потом загрузить уже в стриминговом скрипте (методы save/load - см.документацию)
2. Настоятельно рекомендуется сначала отладить весь процесс чтения данных, предсказания и сохранения результатов в **батчевом** режиме (сгенерить порцию событий чекером и потом считать ее в обычный DataFrame через  `spark.read.format("kafka")`. И уже только после проверки полной  работоспособсности всего скрипта и корректности формата выходных данных в Кафке, переписать на стриминговый режим (благодаря единому API Spark, это не потребует больших изменений)

Обязательное условие зачета лабораторной работы – это выкладка после дедлайна лабы своего решения в репозиторий. Как это сделать, можно прочитать [здесь](/git.md). Если будут вопросы – спрашивайте в Telegram.


In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 5 --executor-memory 4g --driver-memory 3g pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType, DoubleType, FloatType
from pyspark.sql.functions import udf

from pyspark.ml.feature import Tokenizer, HashingTF, StopWordsRemover, IDF, Normalizer, Imputer, VectorAssembler, MinMaxScaler, CountVectorizer
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

import numpy as np
import re
import itertools
from pyspark.ml import Pipeline


conf = SparkConf()
conf.set("spark.app.name", "Dmitry Zh app") 

spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [171]:
def kill_all():
    streams = SparkSession.builder.getOrCreate().streams.active
    if streams:
        for s in streams:
            desc = s.lastProgress["sources"][0]["description"]
            s.stop()
            print("Stopped {s}".format(s=desc))

In [ ]:
def create_parquet_sink(df, file_name):
    return df \
            .repartition(1) \
            .writeStream \
            .format("parquet") \
            .option("path", "{f}".format(f=file_name)) \
            .option("checkpointLocation", "/tmp/{f}".format(f=file_name)) \
            .trigger(processingTime="10 seconds")

In [210]:
!hdfs dfs -ls -h /tmp/check_dmitry_zh

ls: `/tmp/check_dmitry_zh': No such file or directory


In [172]:
kill_all()

In [4]:
import pyspark.sql.types as t


path = "/labs/slaba04/gender_age_dataset.txt"

schema = t.StructType(fields=[
    t.StructField("gender", t.StringType()),
    t.StructField("age", t.StringType()),
    t.StructField("uid", t.StringType()),
    t.StructField("user_json", t.StringType()),
])    

train_data = spark.read.csv(path, header=True, schema=schema, sep="\t")

In [6]:
train_data.show(10, False)

+------+-----+------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [33]:
visits_schema = t.StructType([
    t.StructField("visits", t.ArrayType(\
            t.StructType(\
                [t.StructField('url', t.StringType(), True),\
                 t.StructField('timestamp', t.LongType(), True)])))
])

In [145]:
from pyspark.sql.window import Window
df  = (
    train_data.select(
        f.col("gender"), f.col("age"), f.col("uid"),
        f.explode(f.from_json(f.col("user_json"), visits_schema).getItem("visits").getItem("url")).alias("url")
    )
    .selectExpr("gender", "age", "uid", "parse_url(url,'HOST') as url")
    .groupBy("gender", "age", "uid").agg(f.collect_list("url").alias("url"))
    .where("gender != '-' and age != '-' and size(url)!=0")
    .select("gender", "age", "uid" 
            , f.dense_rank().over(Window.orderBy(f.concat_ws(" ", f.col("gender"), f.col("age")))).alias("category")
            , "url")
).repartition(10).cache()

In [146]:
df.rdd.getNumPartitions()


10

In [147]:
df.select("gender", "age", "category").distinct().show(50 ,False)

+------+-----+--------+
|gender|age  |category|
+------+-----+--------+
|F     |25-34|2       |
|F     |45-54|4       |
|F     |35-44|3       |
|M     |35-44|8       |
|M     |>=55 |10      |
|F     |18-24|1       |
|M     |45-54|9       |
|M     |25-34|7       |
|M     |18-24|6       |
|F     |>=55 |5       |
+------+-----+--------+



In [148]:
train = df.sampleBy("category", fractions={1: 0.8, 2: 0.8, 3: 0.8, 4: 0.8, 5: 0.8, 6: 0.8, 7: 0.8, 8: 0.8, 9: 0.8, 10: 0.8}, seed=42).cache()
test = df.join(train, on=["uid", "gender", "age"], how="leftanti").coalesce(10).cache()

In [150]:
train.count()

28800

In [154]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="category", metricName="accuracy")
tf = HashingTF(numFeatures=10000, binary=True, inputCol="url", outputCol="url_tf")

rfc = RandomForestClassifier(featuresCol=tf.getOutputCol(), labelCol="category")

In [155]:
paramGrid = ParamGridBuilder().build() 

pipeline = Pipeline(stages=[
    tf,
    rfc
])

crossval = CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid,
                              evaluator=evaluator, numFolds=2, parallelism=2)
model = crossval.fit(train)
predictions = model.transform(test)

In [156]:
evaluator.evaluate(predictions)

0.23337988826815642

In [159]:
predictions.select("prediction").distinct().show(10, False)

+----------+
|prediction|
+----------+
|8.0       |
|7.0       |
|3.0       |
|2.0       |
+----------+



In [160]:
KAFKA_BOOTSTRAP_SERVER = 'spark-master-1.newprolab.com:6667'
KAFKA_INPUT_TOPIC = 'input_dmitriy.zhuykov'
KAFKA_OUTPUT_TOPIC = 'dmitriy.zhuykov'

In [162]:
read_kafka_params = {
    "kafka.bootstrap.servers": KAFKA_BOOTSTRAP_SERVER,
    "subscribe": KAFKA_INPUT_TOPIC,
    "startingOffsets": "earliest",
    "failOnDataLoss": "False"
}

kafka_sdf = spark.read.format("kafka").options(**read_kafka_params).load().cache()

In [163]:
event_schema = (t.StructType(
                    [t.StructField('uid', t.StringType(), True)
                   , t.StructField('visits', t.StringType(), True)]))

In [164]:
visit_schema = (t.ArrayType(t.StructType([t.StructField('url', t.StringType(), True)
                   , t.StructField('timestamp', t.LongType(), True)])))

In [181]:
clean_df = (kafka_sdf.select(f.col('value').cast('string').alias('value'))
                     .select(f.from_json(f.col('value'), event_schema).alias('event'))
                     .select('event.uid', f.from_json(f.col('event.visits'), visit_schema).alias('visits'))
)
clean_df.count()

5000

In [182]:
clean_df = (clean_df.select("uid", f.explode(f.col("visits").getItem("url")).alias("url"))
            .selectExpr("uid", "parse_url(url,'HOST') as url")
    .groupBy("uid").agg(f.collect_list("url").alias("url")).coalesce(10))
clean_df.show(5, False)

+------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|uid                                 |url                                                                                                                                                                                                                                                                                                      

In [183]:
clean_df.rdd.getNumPartitions()

10

In [177]:
predictions_df = model.transform(clean_df)

In [178]:
predictions_df.show(5, False)

+------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------

In [189]:
predictions_df = (predictions_df.join(
    df.select("gender", "age", f.col("category").alias("prediction")).distinct(), how='left', on='prediction')
                  .coalesce(10))

In [190]:
predictions_df.rdd.getNumPartitions()

10

In [193]:
predictions_df.count()

5000

In [192]:
predictions_df.select("gender", "age").select([f.count(f.when(f.isnan(c) | f.col(c).isNull(), c)).alias(c) for c in predictions_df.select("gender", "age").columns]).show()

+------+---+
|gender|age|
+------+---+
|     0|  0|
+------+---+



In [194]:
predictions_df = predictions_df.select("uid", "gender", "age")

In [195]:
kafka_out_df = (predictions_df.select(f.to_json(f.struct(*predictions_df.columns)).alias('value')))

In [197]:
(
    kafka_out_df
    .write
    .format('kafka')
    .option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVER)
    .option('topic', KAFKA_OUTPUT_TOPIC)
    .save()
)

In [198]:
kafka_out_df.show(5, False)

+-------------------------------------------------------------------------+
|value                                                                    |
+-------------------------------------------------------------------------+
|{"uid":"0108d217-e476-493d-8c81-a9744f12451a","gender":"M","age":"25-34"}|
|{"uid":"0192cc54-559c-4c8e-89b4-5f4bf31e4245","gender":"M","age":"25-34"}|
|{"uid":"019acd5e-be9a-4cde-8280-0d2376dc016d","gender":"M","age":"25-34"}|
|{"uid":"02e7f830-da57-4d57-a916-10ff45f5de7b","gender":"M","age":"25-34"}|
|{"uid":"1d160259-73d8-4514-911e-757042b22812","gender":"M","age":"25-34"}|
+-------------------------------------------------------------------------+
only showing top 5 rows



### Stream

In [203]:
!hdfs dfs -ls -h /tmp

Found 2236 items
drwxr-xr-x   - grigoriy.bomeyko         hdfs          0 2022-10-23 16:27 /tmp/1
drwxr-xr-x   - polina.berezhnaya        hdfs          0 2022-08-15 21:28 /tmp/6erezhnaya
-rw-r--r--   3 savva.yugov              hdfs      6.6 K 2022-01-18 14:30 /tmp/README
drwxr-xr-x   - teacher2                 hdfs          0 2022-10-04 17:46 /tmp/a2.json
drwxr-xr-x   - teacher2                 hdfs          0 2022-10-04 17:46 /tmp/a2.orc
drwxr-xr-x   - teacher2                 hdfs          0 2022-10-04 17:46 /tmp/a2.parquet
drwxr-xr-x   - teacher2                 hdfs          0 2022-10-04 21:07 /tmp/a3.json
drwxr-xr-x   - teacher2                 hdfs          0 2022-10-04 21:07 /tmp/a3.orc
drwxr-xr-x   - teacher2                 hdfs          0 2022-10-04 21:07 /tmp/a3.parquet
drwxr-xr-x   - artem.lyashenko          hdfs          0 2022-07-19 15:35 /tmp/aba02_domains.txt
-rw-r--r--   3 mikhail.grichik          hdfs         93 2022-03-25 18:52 /tmp/airport-codes-mini.csv
-rw-r--r--  

drwxr-xr-x   - ilya.nechaev             hdfs          0 2022-07-25 23:58 /tmp/temporary-859ffa32-4b29-4ce9-bf8e-9c7cb571a6e5
drwxr-xr-x   - nikita.ryzhov            hdfs          0 2022-10-22 10:08 /tmp/temporary-85eb8fd2-625c-4664-8d14-75968c2a4464
drwxr-xr-x   - vyacheslav.kuzevanov     hdfs          0 2022-07-31 13:41 /tmp/temporary-863cc464-cf73-4319-a025-b7e3d6e7b97a
drwxr-xr-x   - svetlana.lapina          hdfs          0 2022-02-17 14:41 /tmp/temporary-867a5a8e-a7a0-48d3-abb8-004208717d67
drwxr-xr-x   - anton.mikhalev           hdfs          0 2022-10-18 23:56 /tmp/temporary-86b0c06a-dab1-4694-b89e-6b83bcbcad08
drwxr-xr-x   - nikita.redin             hdfs          0 2022-04-17 09:57 /tmp/temporary-87049227-fd71-451a-a39b-46c41f2630ec
drwxr-xr-x   - igor.shuvalov            hdfs          0 2022-07-28 21:29 /tmp/temporary-87242087-3289-47d0-8151-6df4a5b849f1
drwxr-xr-x   - alexander.tsvetkov       hdfs          0 2022-04-17 11:25 /tmp/temporary-874828b1-96e3-455f-8704-e4fb98125299


In [205]:
kafka_stream = (
    spark
    .readStream
    .format('kafka')
    .option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVER)
    .option('subscribe', KAFKA_INPUT_TOPIC)
    .option('startingOffsets', "earliest")
    .option('failOnDataLoss', 'False')
    .load()
)

In [207]:
event_schema = (t.StructType(
                    [t.StructField('uid', t.StringType(), True)
                   , t.StructField('visits', t.StringType(), True)]))

visit_schema = (t.ArrayType(t.StructType([t.StructField('url', t.StringType(), True)
                   , t.StructField('timestamp', t.LongType(), True)])))

clean_df = (kafka_stream.select(f.col('value').cast('string').alias('value'))
                     .select(f.from_json(f.col('value'), event_schema).alias('event'))
                     .select('event.uid', f.from_json(f.col('event.visits'), visit_schema).alias('visits'))
)

clean_df = (clean_df.select("uid", f.explode(f.col("visits").getItem("url")).alias("url"))
            .selectExpr("uid", "parse_url(url,'HOST') as url")
    .groupBy("uid").agg(f.collect_list("url").alias("url")).coalesce(10))

predictions_df = model.transform(clean_df)

predictions_df = (predictions_df.join(
    df.select("gender", "age", f.col("category").alias("prediction")).distinct(), how='left', on='prediction')
                  .coalesce(10))

predictions_df = predictions_df.select("uid", "gender", "age")

In [208]:
kafka_write_stream = (
    predictions_df.select(f.to_json(f.struct(*predictions_df.columns)).alias('value'))
    .writeStream
    .format('kafka')
    .outputMode('append')
    .option('checkpointLocation', "/tmp/check_dmitry_zh")
    .option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVER)
    .option('topic', KAFKA_OUTPUT_TOPIC)


)

In [ ]:
kill_all()

In [ ]:
sc.stop()